# Orchestrator
## Clasificador de transacciones fraudulentas
### Importacion de librerias

In [2]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl, Input, MLClient
from pprint import pprint
from azure.ai.ml.entities import ResourceConfiguration
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

### Inicio de sesión
Ya que estamos usando el presente notebook desde Azure, podemos acceder a nuestras credenciales con unas cuentas lineas de código.

In [3]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: /config.json


### Creación de directorios
Procedemos a crear los directorios donde estarán los datasets tanto para pruebas como para validación de nuestro modelo.

In [3]:
import os

directory_for_training = "training-ml-table"
directory_for_validation = "validation-ml-table"

try:
    os.mkdir(directory_for_training)
    os.mkdir(directory_for_validation)  
except:
    print("The directories already exist.")

The directories already exist.


### Distribución de datos
Se guardara en los directorios creados los dataset correspondientes siguiendo una distribución de 80% para pruebas y 20% para validación.
A continuación crearemos dos dataframes con los datos tanto para pruebas como validación de cada dataset. Es decir, crearemos 4 sub dataset que provienen de los 2 que tenemos conocidos como atípicos y no atípicos.

Queremos agregar lo importante que es declarar que no haya indices al momento de crear los csv separados ya que con entrenamientos anteriores, cuando queríamos hacer las inferencias de los modelos ya deployados nos pedía un `Column2` el cual hacia referencia a una columna sin nombre, es decir, al índice guardado en el csv. Por culpa de este pequeño error tuvimos que correr todo el orquestador desde el inicio junto al procesamiento de modelos, endpoint y deploys que conlleva.

In [4]:
def save_dataset_for_train_and_validation(file_name, test_size, csv_name):
    df_credit_card_atypical = pd.read_csv(file_name, index_col=0)

    X = df_credit_card_atypical.iloc[:, :-1 ]
    y = df_credit_card_atypical.iloc[:, -1 ]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)

    creditcard_atypical_train_df = pd.merge(X_train, y_train, left_index=True, right_index=True, how="inner")
    creditcard_atypical_validation_df = pd.merge(X_test, y_test, left_index=True, right_index=True, how="inner")

    creditcard_atypical_train_df.to_csv(f'./{directory_for_training}/train_data_{csv_name}.csv',index=False)
    creditcard_atypical_validation_df.to_csv(f'./{directory_for_validation}/valid_data_{csv_name}.csv',index=False)

También queremos aclarar que ya no haremos un entrenamiento con los datos atípicos ya que los entrenamientos anteriores con los no atípicos nos dieron valores de accuracy que tienden a 1, por lo cual consideramos que es innecesario y de esta manera también ahorramos un poco de recursos.

In [5]:
#save_dataset_for_train_and_validation('creditcard_atypical.csv', 0.2, 'atypical')
save_dataset_for_train_and_validation('creditcard_non_atypical.csv', 0.2, 'non_atypical')

### Creación de MLTables
Ahora procederemos a la creación de MLTables para referencias nuestros dataset de entrenamiento y validación para nuestro modelo.

In [6]:
import mltable

paths_train = [
    {'file': './training-ml-table/train_data_non_atypical.csv'}
]
paths_validation = [
    {'file': './validation-ml-table/valid_data_non_atypical.csv'}
]

train_table = mltable.from_delimited_files(paths_train)
train_table.save('./training-ml-table')

train_table = mltable.from_delimited_files(paths_validation)
train_table.save('./validation-ml-table')

### Definición de parámetros del Job
Ahora definiremos donde están los MLTables previamente creados para luego, una vez guardados, definir el Job donde usaremos un cluster de la serie D11-15 v2 del tipo `Standard_DS11_v2`
#### ¿Por qué escogimos esa computadora?
Inicialmente planeábamos usar una `Standard_DS2_v2` ya que es una General purpose y para nosotros eso era sinónimo de mas barato, pero indagando mas sobre las demás opciones y teniendo en cuenta el ejemplo en el repositorio de Azure, buscamos mas sobre las serie D, específicamente el modelo `STANDARD_DS12_V2` el cual nos llamo la atención por su precio y la gran cantidad de memoria Ram que ofrece (Ideal para nosotros teniendo en cuenta que tenemos un dataset grande). Por parte del precio nos dimos cuenta que en realidad cobran por la cantidad de cores, siendo la mas barata la `STANDARD_DS11_V2` por 0.185/hora con 2 cores y 14 GB de Ram, y teniendo en cuenta que nuestro Job correrá un máximo de 30 minutos decidimos que no queríamos gastar mas de 1 dólar en el entrenamiento ya que en las pruebas anteriores ya habíamos acumulado mas de 15 dólares por lo cual teniendo en cuenta que una PC cuesta 0.185 la hora, seria 0.0925 para media hora de uso, por lo tanto, la cantidad de nodos para gastar 1 dólar en media hora seria de:

$$cantidad = \frac{1}{0.0925} \approx 5 nodos$$

En resumen, elegimos el PC `STANDARD_DS11_V2` por su gran capacidad de ram (Ideal para nuestro dataset) y su precio accesible. Se usara un máximo de 5 nodos.

También declaramos que la columna objetivo es Class ya que es nuestra variable categórica la cual clasifica que transacción es fraudulenta y cual no.

Todo lo previamente declaro podemos verlo como entradas, salidas, tiempo y recursos de la siguiente manera:
- **Entrada**. Dataset de entrenamiento (`my_training_data_input`) y de validación (`my_validation_data_input`)
- **Salida**. Modelos ordenados por la métrica principal definida `accuracy`
- **Tiempo**. Definimos un tiempo máximo de ejecución de 30 minutos en `exp_timeout` 
- **Recursos**. Cluster con el nombre de 'DS11-v2-cpu-cluster' del CPU del tipo `Standard_DS11v2`

In [7]:
training_mltable_path = "./training-ml-table/"
validation_mltable_path = "./validation-ml-table/"

my_training_data_input = Input(type=AssetTypes.MLTABLE, path=training_mltable_path)
my_validation_data_input = Input(type=AssetTypes.MLTABLE, path=validation_mltable_path)

Aquí definimos el cluster, como dijimos anteriormente, usaremos `STANDARD_DS11_V2` con un tiempo de inactividad de 120 segundos para que en caso de no estar usándose en ese lapso de tiempo, los nodos se marquen como inactivos para dejar de incurrir en costos pasados 2 minutos y 5 instancias como máximos ya que es lo que definimos anteriormente para no tener un costo superior a 1 dólar por la duración maxima de ejecución del Job.

In [8]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "DS11-v2-cpu-cluster"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="STANDARD_DS11_V2",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=5,
    )
    ml_client.begin_create_or_update(compute_config).result()

Found existing compute target.


In [9]:
exp_name = 'creditcard-fraud-classification'
exp_timeout = 30

fraud_classification_job = automl.classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="Class",
    primary_metric="accuracy",
    tags={"classification": "2"},
)

### Creación del Job
Ahora procedemos a ejecutar el Jobs previamente definido y también le iniciamos el seguimiento en la segunda celda.

In [10]:
returned_job = ml_client.jobs.create_or_update(
    fraud_classification_job
)  # submit the job to the backend

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-instance-jupyter/code/Users/carrasco.juan12/training-ml-table' 'https://creditcardml5040471872.blob.core.windows.net/azureml-blobstore-9dcd07be-3c9d-4e45-b0d3-2db95e9e2c49/LocalUpload/0ae6e8cf1cf3d01b37e3772236b07ac5/training-ml-table' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading training-ml-table (155.05 MBs): 100%|██████████| 155050017/155050017 [00:01<00:00, 93714082.35it/s]


Uploading validation-ml-table (38.77 MBs): 100%|██████████| 38765324/38765324 [00:04<00:00, 8108082.61it/s] 




In [11]:
ml_client.jobs.stream(returned_job.name)

RunId: khaki_planet_gyy3vkmj7n
Web View: https://ml.azure.com/runs/khaki_planet_gyy3vkmj7n?wsid=/subscriptions/9d6f3686-6c64-4aea-8d5a-3cd7cb82619b/resourcegroups/ml-proyecto/workspaces/creditcard-ml

Execution Summary
RunId: khaki_planet_gyy3vkmj7n
Web View: https://ml.azure.com/runs/khaki_planet_gyy3vkmj7n?wsid=/subscriptions/9d6f3686-6c64-4aea-8d5a-3cd7cb82619b/resourcegroups/ml-proyecto/workspaces/creditcard-ml



## Deploy
### Guardado del tracking URI
Procedemos a guardar el tracking URI de nuestro Job para hacerle seguimiento usando MLflow para de esa manera obtener el mejor modelo obtenido por AutoML.

In [12]:
import mlflow

MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/9d6f3686-6c64-4aea-8d5a-3cd7cb82619b/resourceGroups/ML-proyecto/providers/Microsoft.MachineLearningServices/workspaces/CreditCard-ML


Obtener el Job que entreno los modelos usando AutoML.

In [13]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [14]:
from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts

# Initialize MLFlow client
mlflow_client = MlflowClient()

### Resultados del Job
Como podemos observar, al llamar a job y mostrar lo que corrió, podemos notar muchas métricas además de los modelos que usando junto a sus resultados de acuerdo a la métrica principal definida. En nuestro caso, la métrica principal es el accuracy y podemos ver los resultados en el campo campo de `score_000`. Como también se puede notar es demasiada información, por lo cual en la siguiente celda mostraremos mas ordenadamente los modelos probados con su respetiva métrica.

In [15]:
job = ml_client.jobs.get(name=returned_job.name)
mlflow_parent_run = mlflow_client.get_run(job.name)
print(mlflow_parent_run)

<Run: data=<RunData: metrics={'AUC_macro': 0.9992520151166466,
 'AUC_micro': 0.9992486184022564,
 'AUC_weighted': 0.9992520151166466,
 'accuracy': 0.9980947577820739,
 'average_precision_score_macro': 0.9988802694638901,
 'average_precision_score_micro': 0.9988756021883952,
 'average_precision_score_weighted': 0.9988799124439014,
 'balanced_accuracy': 0.9980929232495093,
 'f1_score_macro': 0.9980947421192317,
 'f1_score_micro': 0.9980947577820739,
 'f1_score_weighted': 0.9980947473741918,
 'log_loss': 0.027649516098839436,
 'matthews_correlation': 0.996196730447637,
 'norm_macro_recall': 0.9961858464990185,
 'precision_score_macro': 0.9981038072575847,
 'precision_score_micro': 0.9980947577820739,
 'precision_score_weighted': 0.9981019831950062,
 'recall_score_macro': 0.9980929232495093,
 'recall_score_micro': 0.9980947577820739,
 'recall_score_weighted': 0.9980947577820739,
 'weighted_accuracy': 0.9980965923112435}, params={}, tags={'automl_best_child_run_id': 'khaki_planet_gyy3vkmj7n

### Modelos probados
Ahora podemos visualizar tanto los modelos como los pre procesadores usados, cada uno de ellos están ordenados de tal manera que el primer elemento de la lista de modelos pertenece al primer elemento de la lista de pre procesadores y asi sucesivamente. Es por esta razón que mostramos los datos ordenados en un dataframe.
Para entender mejor la recopilación de los datos podemos verlo de la siguiente manera:
- Modelos. Esta denotado con la etiqueta de `run_algorithm_000`.
- Preprocessor. Esta denotado con la etiqueta de `run_preprocessor_000`.
- Métrica principal. Esta denotado con la etiqueta de `score_000`. En nuestro caso, como podemos observar arriba, la métrica primaria fue definida como `accuracy` por lo cual los datos que representa `score_000` es el accuracy de cada modelo.
### El mejor modelo
Como podemos observar en el dataframe, el modelo que tiene el mayor accuracy es un **KNN** con **StandardScalerWrapper**. Pero debemos notar que existe una manera mas directa de acceder al mejore modelo y a su vez a sus métricas. Eso lo mostraremos en la siguiente celda.

In [16]:
run_algorithm_000 = mlflow_parent_run.data.tags.get('run_algorithm_000').split(';')
run_preprocessor_000 = mlflow_parent_run.data.tags.get('run_preprocessor_000').split(';')
score_000 = mlflow_parent_run.data.tags.get('score_000').split(';')

accuracy = []
for score in score_000:
    accuracy.append(np.round(float(score),5))

df = pd.DataFrame(index=['Algorithm', 'Preprocessor', 'Accuracy'], data=[run_algorithm_000,run_preprocessor_000,accuracy])
df

,0,1,2,3,4,5,6,7,8
Algorithm,LightGBM,XGBoostClassifier,ExtremeRandomTrees,XGBoostClassifier,KNN,LightGBM,LogisticRegression,LightGBM,KNN
Preprocessor,MaxAbsScaler,MaxAbsScaler,MaxAbsScaler,SparseNormalizer,StandardScalerWrapper,MaxAbsScaler,RobustScaler,MaxAbsScaler,StandardScalerWrapper
Accuracy,0.99227,0.99798,0.93528,0.95881,0.99809,0.97504,0.95689,0.95801,0.99761


### Obtener el mejor modelo
Para lograr esto debemos usar la etiqueta de `automl_best_child_run_id` el cual nos referenciara a las métricas del mejor modelo como podemos observar a continuación.

In [17]:
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
best_run = mlflow_client.get_run(best_child_run_id)
best_run.data.metrics

{'average_precision_score_macro': 0.9988802694638901,
 'weighted_accuracy': 0.9980965923112435,
 'AUC_macro': 0.9992520151166466,
 'recall_score_macro': 0.9980929232495093,
 'balanced_accuracy': 0.9980929232495093,
 'recall_score_weighted': 0.9980947577820739,
 'AUC_micro': 0.9992486184022564,
 'f1_score_macro': 0.9980947421192317,
 'precision_score_micro': 0.9980947577820739,
 'f1_score_weighted': 0.9980947473741918,
 'precision_score_weighted': 0.9981019831950062,
 'AUC_weighted': 0.9992520151166466,
 'norm_macro_recall': 0.9961858464990185,
 'average_precision_score_weighted': 0.9988799124439014,
 'matthews_correlation': 0.996196730447637,
 'average_precision_score_micro': 0.9988756021883952,
 'recall_score_micro': 0.9980947577820739,
 'log_loss': 0.027649516098839436,
 'accuracy': 0.9980947577820739,
 'precision_score_macro': 0.9981038072575847,
 'f1_score_micro': 0.9980947577820739}

### Hiperparametros
Ingresado al Job podemos notar que tenemos un parámetro denotado como `params`, lo extraño de esto es que no se tiene ningún otro dato en este apartado sin embargo al usar la interfaz de azure podemos notar que si existen algunos hiperparámetros implementados. Creemos que esto se debe a que nosotros no implementamos hiperparámetros desde código por lo cual no nos los muestra. De todos modos intentamos acceder a los hiperparámetros usando las Id de los modelos generado pero nos encontramos con que no podemos acceder a esos recursos.

In [18]:
mlflow_parent_run.data.params

{}

In [20]:
parent_run_id = mlflow_parent_run.info.run_id
print(parent_run_id)

child_runs = mlflow_client.search_runs(
    experiment_ids=[parent_run_id],
    filter_string=f"tags.mlflow.parentRunId = '{parent_run_id}'"
)
print(child_runs)
for child_run in child_runs:
    print(child_run)
    print('-'*20)

khaki_planet_gyy3vkmj7n
[]


#### Otro metodo
Luego de indagar un poco sobre como funciona el tema de los padres y los hijos con AutoML, pudimos notar que el acceso a los hijos esta bastante limitado según el siguiente [Issue](http://www.limni.net) donde una persona muestra como se pueden acceder a los hijos agregando un numero luego del id del padre, es decir, si el siguiente parent_id definido como `coral_sock_t10p5v595k` entonces para acceder al primer hijo podemos hacer `coral_sock_t10p5v595k_0`. En el siguiente lo demostramos.

In [24]:
i = 1
while True:
    child_run = mlflow_client.get_run(f'{job.name}_{i}')
    if child_run.data.metrics == {}:
        break
    ml_flow_child_name = child_run.data.tags.get('mlflow.runName')
    print(f'Hijo #{i}: {ml_flow_child_name}       \tparams: {child_run.data.params}')
    i += 1

Hijo #1: affable_pasta_zdzt48kr       	params: {}
Hijo #2: ashy_pot_wkv4mlkj       	params: {}
Hijo #3: gifted_pillow_2nbm4cpc       	params: {}
Hijo #4: great_nerve_1yqf8xbp       	params: {}
Hijo #5: ashy_pocket_xb3k1z20       	params: {}
Hijo #6: amusing_cheese_598p8cd9       	params: {}
Hijo #7: red_mango_clpvc2zl       	params: {}
Hijo #8: magenta_date_fzym2nvb       	params: {}


De todas formas, aun que tratamos de ver los params definidos para cada hijo, notamos que todos están vacíos y según lo que leímos en el [Issue](http://www.limni.net) anterior, una de las personas llego a la siguiente conclusion:
> I believe the issue you're encountering with the data.params dictionary being empty is not related to the Azure ML SDK version. I am guessing this seems to be a limitation of the MLflow API in general.

### Descargar del artefacto
Ahora procederemos a descargar el artefacto para hacer pruebas de deployment con otros métodos.

In [25]:
import os

# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)
# Download run's artifacts/outputs
local_path = download_artifacts(
    run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Artifacts downloaded in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-instance-jupyter/code/Users/carrasco.juan12/artifact_downloads/outputs
Artifacts: ['conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'generated_code', 'mlflow-model', 'model.pkl', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_pbi_v_1_0_0.py', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']


In [26]:
os.listdir("./artifact_downloads/outputs/mlflow-model")

['conda.yaml', 'MLmodel', 'model.pkl', 'python_env.yaml', 'requirements.txt']

### Creación del endpoint
Ahora procederemos a crear el endpoint dándole un nombre seguido de la fecha para evitar que existan nombres duplicados ya que de existir nos daría un error en ejecución de nuestro orchestrator. Es importante saber que para nombre existe un limite de 5 caracteres como mínimo y 32 como máximo. Decimos que es importante ya que en nuestra primera ejecución dimos un nombre muy largo lo cual resulto en un error de ejecución, específicamente `BadRequest`.

In [27]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)

# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "cc-classification" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Endpoint to assess the fraudulence of a transaction",
    auth_mode="key",
    tags={"endpoint": "1"},
)
print(online_endpoint_name)

cc-classification10281943835288


In [28]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://cc-classification10281943835288.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://cc-classification10281943835288.eastus.inference.ml.azure.com/swagger.json', 'name': 'cc-classification10281943835288', 'description': 'Endpoint to assess the fraudulence of a transaction', 'tags': {'endpoint': '1'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/9d6f3686-6c64-4aea-8d5a-3cd7cb82619b/resourcegroups/ml-proyecto/providers/microsoft.machinelearningservices/workspaces/creditcard-ml/onlineendpoints/cc-classification10281943835288', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/9d6f3686-6c64-4aea-8d5a-3cd7cb82619b/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:9dcd07be-3c9d-4e45-b0d3-2db95e9e2c49:4965ae3f-a601-42a2-a1f2-2fd18c318c36?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '

### Registro del modelo
Ahora debemos registrar el modelo para hacer deployment. Es por esta razón que usaremos el mejor modelo mandando la id para poder tener el path adecuado asi como también otros datos como el nombre del modelo `model_name`, la descripción `description` y el tipo `type=AssetTypes.MLFLOW_MODEL`.

In [29]:
model_name = "cc-clasification-model"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="Fraudulent transactions classification model",
    type=AssetTypes.MLFLOW_MODEL,
)
registered_model = ml_client.models.create_or_update(model)

### Deployment
Ahora que tenemos tanto el endpoint definido como el modelo registrado, procedemos al deployment ambos definiendo a su vez la instancia sobre la cual estará alojada. Este deploy a su vez también necesita un nombre `name`, el endpoint, el modelo registrado (ambos ya definidos arriba), la instancia `Standard_DS2_v2` la cual tiene un tamaño relativo **X-Small**, elegimos esta instancia ya que consideramos que el costo por tener una instancia funcionando todo tiempo para nuestro deploy puede llevar a costos mas elevados que tenemos a demás de que al ser esta una practica en la cual solo los integrantes del grupo interactuaremos con el endpoint resultante, no necesitaremos mayor fuerza de procesamiento. Por otro lado, al ejecutar nos sale un aviso de que la instancia es muy pequeña, consideramos que si es relevante este aviso pero en un caso donde si existan clientes reales que accedan al endpoint resultante, al tener unicamente 2 clientes (los integrantes del grupo) consideramos que no hay de que preocuparnos.

In [30]:
deployment = ManagedOnlineDeployment(
    name="cc-clasification-deploy",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS2_v2",
    instance_count=1)
ml_client.online_deployments.begin_create_or_update(deployment).result()

Instance type Standard_DS2_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint cc-classification10281943835288 exists


..........................................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'cc-classification10281943835288', 'type': 'Managed', 'name': 'cc-clasification-deploy', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/9d6f3686-6c64-4aea-8d5a-3cd7cb82619b/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/od:9dcd07be-3c9d-4e45-b0d3-2db95e9e2c49:2ee057d5-c312-46f7-a0a4-4a4eeeb38481?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/9d6f3686-6c64-4aea-8d5a-3cd7cb82619b/resourceGroups/ml-proyecto/providers/Microsoft.MachineLearningServices/workspaces/creditcard-ml/onlineEndpoints/cc-classification10281943835288/deployments/cc-clasification-deploy', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-instance-jupyter/code/Users/carrasco.juan12', 'creation_context': None, 'serialize': <

#### Inferencias
Ahora que tenemos el endpoint levantado y listo para ser usado, cargamos el dataset del mismo tipo (no atípicos) y le sacamos una muestra fraccionaria del 0.00001 lo cual nos da 5 ejemplos. Una vez que tenemos esos ejemplos procedemos a separar en los datos de entrada `df_data` y los resultados esperados `list_result`.

In [6]:
df_creditcard = pd.read_csv('creditcard_non_atypical.csv', index_col=0)
df_sample_data = df_creditcard.sample(frac=0.00001, replace=True, random_state=1)
print(f'Sample shape data for testing the endpoint: {df_sample_data.shape}')
print(f'Columns: {df_creditcard.columns}')

df_data = df_sample_data.iloc[:,:-1]
list_result = df_sample_data.iloc[:,-1].to_list()
print(f'Data shape: {df_data.shape} Expected result shape: {len(list_result)}')

Sample shape data for testing the endpoint: (5, 10)
Columns: Index(['V14', 'V12', 'V3', 'V10', 'V9', 'V16', 'V1', 'V11', 'V4', 'Class'], dtype='object')
Data shape: (5, 9) Expected result shape: 5


Ahora que tenemos los datos separados, ingresamos a la información de nuestro endpoint para recopilar el código base para realizar las inferencias. Dicho código fue parcialmente editado para trabajar con nuestros datos propuestos donde 'data' sera cada fila de los datos de entrada que recopilamos anteriormente y están en formato pandas.dataframe.
Nota: Leyendo la [documentacion](https://learn.microsoft.com/es-es/azure/machine-learning/how-to-deploy-advanced-entry-script?view=azureml-api-1) nos dimos cuenta que el endpoint admite entradas del tipo:
- pandas
- numpy
- pyspark
- Objeto estándar de Python

Sin embargo, decidimos que haremos las inferencias fila por fila para notar también el tiempo de respuesta entre cada llamada.

#### Inferencias usando requests de python
A continuación cargaremos el código necesario separado en funciones tanto para pasar la verificación del lado del cliente, formatear nuestras entradas, crear el request, enviar el request y recibirlo para luego mostrarlo en contraste con las entradas esperadas anteriormente recabadas.

In [7]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True)

In [13]:
def create_endpoint_data(ep_data):
  index = list(range(len(ep_data)))
  data =  {
    "input_data": {
      "columns": [
        "V14",
        "V12",
        "V3",
        "V10",
        "V9",
        "V16",
        "V1",
        "V11",
        "V4"
      ],
      "data": [ep_data]
    }
  }
  return data

def create_request(data):
  body = str.encode(json.dumps(data))
  url = f'https://cc-classification10281943835288.eastus.inference.ml.azure.com/score'
  api_key = 'fKpR8WIthnCVFNY5sxeyw04ScPF6Gh2n'
  if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

  headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'cc-clasification-deploy' }
  req = urllib.request.Request(url, body, headers)
  return req

def make_request(req):
  try:
    response = urllib.request.urlopen(req)

    result = response.read()
    return result
  except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

In [14]:
i = 0
for index, row in df_data.iterrows():
    endpoint_data = create_endpoint_data(row.to_list())
    request = create_request(endpoint_data)
    result = make_request(request)
    print(f'Prediction for row {index} is {result} and the real value is {list_result[i]}')
    i += 1

Prediction for row 136561 is b'[0]' and the real value is 0
Prediction for row 522371 is b'[1]' and the real value is 1
Prediction for row 500324 is b'[1]' and the real value is 1
Prediction for row 521851 is b'[1]' and the real value is 1
Prediction for row 394971 is b'[1]' and the real value is 1


Como podemos observar, todas las inferencias tuvieron una precisión del 100% en este caso, lo cual tiene sentido teniendo en cuenta que tenemos un accuracy muy cercano al 100%.
#### Inferencias usando MLClient
Ahora haremos las inferencias usando ml_cliente para esto usaremos de nuevo la función que pone en formato nuestras entradas para luego crear el archivo `request file` y una vez que tenemos el archivo el archivo creado procederemos a crear el request con la función `make_ml_request`.

Al igual que con la inferencia anterior, tenemos un 100% de exactitud.

#### Nota
Para esta ultima inferencia se opto por hacer hardcode el nombre del endpoint y del deploy ya que la computadora se nos apago y para volver a tener la información cargada en las variables pertinentes tendrías que volver a cargar todo de nuevo, es decir, volver a crear los endpoint, deployment, etc.

In [19]:
request_file_name = 'cc_request_data.json'

def create_request_file(data):
    with open(request_file_name, 'w') as request_file:
        json.dump(data, request_file)

def make_ml_request():
    resp = ml_client.online_endpoints.invoke(
        endpoint_name='cc-classification10281943835288',
        deployment_name='cc-clasification-deploy',
        request_file=request_file_name
    )
    return resp

In [20]:
i = 0
for index, row in df_data.iterrows():
    endpoint_data = create_endpoint_data(row.to_list())
    create_request_file(endpoint_data)
    result = make_ml_request()
    print(f'Prediction for row {index} is {result} and the real value is {list_result[i]}')
    i += 1

Prediction for row 136561 is [0] and the real value is 0
Prediction for row 522371 is [1] and the real value is 1
Prediction for row 500324 is [1] and the real value is 1
Prediction for row 521851 is [1] and the real value is 1
Prediction for row 394971 is [1] and the real value is 1
